In [3]:
from selenium import webdriver 
from bs4 import BeautifulSoup
import time

# 통계지표 검색어를 입력하여, CSV 파일로 저장하기
def download_bok_statistics_by_keyword():
    
    item_found = 0              
    while not item_found:
        
        # 검색어 초기화
        keyword = ""       
        while len(keyword) == 0:
            keyword = str(input("검색할 항목을 입력하세요: "))
        
        # 웹 드라이버 실행
        driver = webdriver.Chrome("./chromedriver.exe")
        driver.implicitly_wait(3)
        driver.get("http://ecos.bok.or.kr/jsp/vis/keystat/#/key") 
        time.sleep(5)             
                   
        items1 = driver.find_elements_by_css_selector('a[class="ng-binding"]')
        items2 = driver.find_elements_by_css_selector('a[class="a-c1-list ng-binding"]')
        items3 = driver.find_elements_by_css_selector('a[class="a-c4-list ng-binding"]')
        driver.implicitly_wait(3)

        items = items1[1:] + items2 + items3    
        
        for idx, item in enumerate(items):
            if keyword in item.text:
                print("검색어 '%s'에 매칭되는 '%s' 통계지표를 검색 중..." % (keyword, item.text))
                item.click()
                item_found = 1
                time.sleep(5)
                break
            elif idx == (len(items) - 1):
                print("검색어 '%s'에 대한 통계지표가 존재하지 않습니다..." % keyword)
                driver.close()
                continue
            else:
                pass
            

    # 검색결과 로딩 HTML 웹 페이지를 파싱 - 통계지표 테이블(표) 양식을 정리
    html_src = driver.page_source
    soup = BeautifulSoup(html_src, 'html.parser')
    driver.close()
  
    table_items = soup.find_all('td', {'class':'ng-binding'})
    date = [t.text for i, t in enumerate(table_items) if i % 3 == 0]
    value = [t.text for i, t in enumerate(table_items) if i % 3 == 1]    
    change = [t.text for i, t in enumerate(table_items) if i % 3 == 2]   
    
    # CSV 파일로 저장
    result_file = open('./data/bok_statistics_%s.csv' % keyword, 'w')
    
    for i in range(len(date)):
        result_file.write("%s, %s, %s" % (date[i], value[i], change[i]))
        result_file.write('\n')
    
    result_file.close()
    print("키워드 '%s'에 대한 통계지표를 저장하였습니다." % keyword)
    
    return date, value, change


#함수 실행 - 'CD수익률' 통계지표를 별도로 검색, CSV 파일로 저장
result = download_bok_statistics_by_keyword()
print(result)

검색할 항목을 입력하세요: CD
검색어 'CD'에 매칭되는 'CD수익률(91일) ('20.08.14)' 통계지표를 검색 중...
키워드 'CD'에 대한 통계지표를 저장하였습니다.
(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020.1', '2020.2', '2020.3', '2020.4', '2020.5', '2020.6', '2020.7', '2020.8.7', '2020.8.10', '2020.8.11', '2020.8.12', '2020.8.13', '2020.8.14'], ['2.72', '2.49', '1.76', '1.49', '1.44', '1.68', '1.69', '1.47', '1.42', '1.23', '1.10', '1.02', '0.79', '0.79', '0.75', '0.74', '0.73', '0.70', '0.67', '0.65'], ['-0.58', '-0.23', '-0.73', '-0.27', '-0.05', '0.24', '0.01', '-0.06', '-0.05', '-0.19', '-0.13', '-0.08', '-0.23', '0.00', '-0.01', '-0.01', '-0.01', '-0.03', '-0.03', '-0.02'])
